In [1]:
import pandas as pd
import datetime
from tqdm import tqdm
from bonds_pricer import Bond, Discount_Factor
import json

import matplotlib.pyplot as plt
import seaborn as sns

# Load the rates

In [2]:
rates = pd.read_excel('/Users/freezylex/Desktop/курсач/cbr_rates.xlsx', parse_dates = True, index_col = 'Date')
rates[0] = rates[0.25]
rates

,0.00,0.25,0.50,0.75,1.00,2.00,3.00,5.00,7.00,10.00,15.00,20.00,30.00
Date,,,,,,,,,,,,,
2015-03-02,0.1354,0.1354,0.1367,0.1380,0.1391,0.1429,0.1452,0.1458,0.1413,0.1303,0.1172,0.1094,0.1015
2015-03-03,0.1241,0.1241,0.1275,0.1307,0.1332,0.1387,0.1398,0.1380,0.1336,0.1258,0.1151,0.1077,0.0990
2015-03-04,0.1277,0.1277,0.1295,0.1312,0.1328,0.1374,0.1394,0.1386,0.1347,0.1276,0.1179,0.1108,0.1019
2015-03-05,0.1266,0.1266,0.1281,0.1296,0.1310,0.1351,0.1365,0.1356,0.1324,0.1266,0.1177,0.1107,0.1019
2015-03-06,0.1211,0.1211,0.1237,0.1261,0.1281,0.1331,0.1350,0.1348,0.1322,0.1269,0.1182,0.1112,0.1020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-02-15,0.0892,0.0892,0.0944,0.0977,0.0997,0.1025,0.1030,0.1031,0.1025,0.1011,0.1003,0.1004,0.1008
2016-02-16,0.0900,0.0900,0.0954,0.0983,0.0999,0.1017,0.1026,0.1033,0.1029,0.1018,0.1010,0.1009,0.1010
2016-02-17,0.0917,0.0917,0.0971,0.0994,0.1003,0.1012,0.1020,0.1024,0.1019,0.1009,0.1003,0.1003,0.1006


In [3]:
rates.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 245 entries, 2015-03-02 to 2016-02-19
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0.0     245 non-null    float64
 1   0.25    245 non-null    float64
 2   0.5     245 non-null    float64
 3   0.75    245 non-null    float64
 4   1.0     245 non-null    float64
 5   2.0     245 non-null    float64
 6   3.0     245 non-null    float64
 7   5.0     245 non-null    float64
 8   7.0     245 non-null    float64
 9   10.0    245 non-null    float64
 10  15.0    245 non-null    float64
 11  20.0    245 non-null    float64
 12  30.0    245 non-null    float64
dtypes: float64(13)
memory usage: 26.8 KB


In [4]:
rates.describe()

,0.00,0.25,0.50,0.75,1.00,2.00,3.00,5.00,7.00,10.00,15.00,20.00,30.00
count,245.000000,245.000000,245.000000,245.00000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000
mean,0.099484,0.099484,0.103653,0.10605,0.107487,0.109710,0.110186,0.109775,0.108842,0.107341,0.104757,0.102364,0.098575
std,0.011302,0.011302,0.009129,0.00826,0.007929,0.008277,0.008750,0.009006,0.008835,0.008211,0.006847,0.006003,0.006538
min,0.084900,0.084900,0.092300,0.09600,0.098100,0.100000,0.099200,0.096800,0.095400,0.095000,0.093900,0.093000,0.085900
25%,0.091000,0.091000,0.097600,0.10100,0.102500,0.103900,0.103200,0.102600,0.102200,0.101300,0.100300,0.098300,0.093000
50%,0.095100,0.095100,0.101200,0.10400,0.105000,0.107600,0.108700,0.108900,0.108200,0.106600,0.103700,0.100600,0.097500
75%,0.103700,0.103700,0.104200,0.10640,0.109500,0.113200,0.113700,0.113200,0.112400,0.111100,0.108500,0.105400,0.103000
max,0.135400,0.135400,0.136700,0.13800,0.139100,0.142900,0.145200,0.145800,0.141300,0.132700,0.128000,0.122300,0.112400


# Load json data

In [5]:
file = open('/Users/freezylex/Desktop/курсач/bonds_info.json', 'r')
data_bonds = json.load(file)
data_bonds

{'SU26214RMFS': {'payments': ['04.12.2013',
   '04.06.2014',
   '03.12.2014',
   '03.06.2015',
   '02.12.2015',
   '01.06.2016',
   '30.11.2016',
   '31.05.2017',
   '29.11.2017',
   '30.05.2018',
   '28.11.2018',
   '29.05.2019',
   '27.11.2019',
   '27.05.2020'],
  'coupon_size': 6.4,
  'sum_of_coupon': [31.91,
   31.91,
   31.91,
   31.91,
   31.91,
   31.91,
   31.91,
   31.91,
   31.91,
   31.91,
   31.91,
   31.91,
   31.91,
   31.91],
  'redemption_size': 1000,
  'issue_date': '05.06.2013',
  'times_per_year': 2,
  'redemption_date': '27.05.2020'},
 'SU26215RMFS': {'payments': ['26.02.2014',
   '27.08.2014',
   '25.02.2015',
   '26.08.2015',
   '24.02.2016',
   '24.08.2016',
   '22.02.2017',
   '23.08.2017',
   '21.02.2018',
   '22.08.2018',
   '20.02.2019',
   '21.08.2019',
   '19.02.2020',
   '19.08.2020',
   '17.02.2021',
   '18.08.2021',
   '16.02.2022',
   '17.08.2022',
   '15.02.2023',
   '16.08.2023'],
  'coupon_size': 7.0,
  'sum_of_coupon': [33.56,
   34.9,
   34.9,
   

# Bond pricing

In [6]:
def make_payments(json_file, ticker):
    index = pd.to_datetime(json_file[ticker]['payments'], format='%d.%m.%Y')
    coupons = json_file[ticker]['sum_of_coupon']
    coupons[-1] += json_file[ticker]['redemption_size']
    payments = pd.Series(index=index,
                         data=coupons)
    return payments

In [7]:
cols = list(data_bonds.keys())
cols.remove('SU26217RMFS')
cols.remove('SU26218RMFS')

In [8]:
priced_bonds = pd.DataFrame(columns = cols,
                           index = rates.index)
priced_bonds

,SU26214RMFS,SU26215RMFS,SU26216RMFS,SU26203RMFS,SU26204RMFS,SU26205RMFS,SU26206RMFS,SU26207RMFS,SU26208RMFS,SU26209RMFS,SU26210RMFS,SU26211RMFS,SU26212RMFS
Date,,,,,,,,,,,,,
2015-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import numpy as np

In [10]:
def price(ticker):
    global data_bonds
    global rates
    global priced_bonds

    payments = make_payments(data_bonds, ticker)

    coupon = data_bonds[ticker]['coupon_size']
    times = data_bonds[ticker]['times_per_year']
    issue_date = data_bonds[ticker]['issue_date']
    redemp_date = data_bonds[ticker]['redemption_date']

    year_finish, month_finish, day_finish = ([int(el) for el in redemp_date.split('.')][::-1])
    year_start, month_start, day_start = ([int(el) for el in issue_date.split('.')][::-1])

    start = datetime.datetime(year_start, month_start, day_start)
    end = datetime.datetime(year_finish, month_finish, day_finish)

    bond = Bond(coupons=coupon,
                times_per_year=times,
                face_value=1000,
                issue_date=start,
                redemption_date=end,
                payments=payments)

    def return_difference(now, red_date):
        now = pd.to_datetime(now)
        red_date = pd.to_datetime(red_date)
        delta = (red_date - now) / datetime.timedelta(days=365)
        return delta

    payment_1_per = bond.payments.values[1]
    prices = []
    for date in tqdm(rates.index):
        now = rates.loc[date].name.date()
        diff_in_years = return_difference(now, bond.redemption_date)
        max_diff = np.max(diff_in_years)
        index = np.argwhere(rates.columns > max_diff)[0]
        rates_to_take = rates.columns[:int(index) + 1]
        rates_dis = rates[rates_to_take].loc[date]
        df = Discount_Factor(rates=rates_dis,
                             date=datetime.datetime(now.year, now.month, now.day),
                             curr_bond=bond,
                             payment=payment_1_per)
        prices.append(df.price_bond())
    priced_bonds[ticker] = prices

In [11]:
for col in cols:
    price(col)

100%|██████████| 245/245 [00:01<00:00, 140.75it/s]


In [12]:
priced_bonds

,SU26214RMFS,SU26215RMFS,SU26216RMFS,SU26203RMFS,SU26204RMFS,SU26205RMFS,SU26206RMFS,SU26207RMFS,SU26208RMFS,SU26209RMFS,SU26210RMFS,SU26211RMFS,SU26212RMFS
Date,,,,,,,,,,,,,
2015-03-02,723.529917,683.709950,772.563851,915.676595,843.449433,746.692677,876.778472,724.669423,805.446744,724.566505,753.355897,690.181056,655.484705
2015-03-03,744.838613,707.396116,789.387760,921.681293,854.633905,770.673184,884.549534,746.412025,821.407983,750.056194,773.164913,714.588869,674.886615
2015-03-04,743.406678,703.008227,789.181296,922.643435,855.583342,768.461685,886.324514,738.453057,821.468002,746.646955,772.232948,710.729571,666.346842
2015-03-05,751.807997,710.336991,796.688487,925.021155,861.554664,777.157860,890.498966,744.380832,828.804164,754.953792,780.438605,718.481981,671.405216
2015-03-06,754.440057,711.266200,799.851437,928.040913,864.937058,779.431205,893.847750,744.347191,832.083399,756.451837,783.356500,719.673454,671.092255
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-02-15,875.549761,844.911638,909.353935,989.882758,954.222718,903.326142,970.491715,884.794383,935.072245,887.529013,899.124573,852.104702,804.090234
2016-02-16,875.766706,843.386570,910.220040,989.553629,955.505602,902.855490,970.672822,881.682674,936.029889,886.506982,899.669049,850.847310,800.928230
2016-02-17,878.054006,847.524101,911.731373,988.926393,956.337924,905.861173,970.572853,886.587104,937.395774,890.245047,901.602319,854.822008,805.613328


In [14]:
priced_bonds = priced_bonds.loc['2015-05-30':]
priced_bonds

,SU26214RMFS,SU26215RMFS,SU26216RMFS,SU26203RMFS,SU26204RMFS,SU26205RMFS,SU26206RMFS,SU26207RMFS,SU26208RMFS,SU26209RMFS,SU26210RMFS,SU26211RMFS,SU26212RMFS
Date,,,,,,,,,,,,,
2015-06-01,841.871101,814.116035,877.637095,962.360559,927.703310,874.510470,944.517905,855.716252,907.023583,858.495079,867.291352,821.476399,775.419882
2015-06-02,843.133216,816.087049,878.336214,962.624957,927.982547,876.057278,944.758503,857.960301,907.617733,860.441234,868.293179,823.428826,777.625125
2015-06-03,837.940174,810.518494,874.314462,962.359515,925.543738,870.404759,943.827079,852.567672,903.784692,854.626169,863.592439,817.782784,772.745829
2015-06-04,828.564433,797.254662,866.455622,960.886867,919.786428,859.540276,939.895888,835.702836,896.152959,842.257981,854.812542,805.102427,756.413509
2015-06-05,830.765402,796.897664,868.102852,961.735199,920.660148,861.277549,939.930685,832.361564,897.706827,842.980767,856.778693,805.304501,752.553516
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-02-15,875.549761,844.911638,909.353935,989.882758,954.222718,903.326142,970.491715,884.794383,935.072245,887.529013,899.124573,852.104702,804.090234
2016-02-16,875.766706,843.386570,910.220040,989.553629,955.505602,902.855490,970.672822,881.682674,936.029889,886.506982,899.669049,850.847310,800.928230
2016-02-17,878.054006,847.524101,911.731373,988.926393,956.337924,905.861173,970.572853,886.587104,937.395774,890.245047,901.602319,854.822008,805.613328


In [15]:
priced_bonds = priced_bonds/10
priced_bonds.head()

,SU26214RMFS,SU26215RMFS,SU26216RMFS,SU26203RMFS,SU26204RMFS,SU26205RMFS,SU26206RMFS,SU26207RMFS,SU26208RMFS,SU26209RMFS,SU26210RMFS,SU26211RMFS,SU26212RMFS
Date,,,,,,,,,,,,,
2015-06-01,84.187110,81.411603,87.763710,96.236056,92.770331,87.451047,94.451790,85.571625,90.702358,85.849508,86.729135,82.147640,77.541988
2015-06-02,84.313322,81.608705,87.833621,96.262496,92.798255,87.605728,94.475850,85.796030,90.761773,86.044123,86.829318,82.342883,77.762513
2015-06-03,83.794017,81.051849,87.431446,96.235951,92.554374,87.040476,94.382708,85.256767,90.378469,85.462617,86.359244,81.778278,77.274583
2015-06-04,82.856443,79.725466,86.645562,96.088687,91.978643,85.954028,93.989589,83.570284,89.615296,84.225798,85.481254,80.510243,75.641351
2015-06-05,83.076540,79.689766,86.810285,96.173520,92.066015,86.127755,93.993068,83.236156,89.770683,84.298077,85.677869,80.530450,75.255352


In [16]:
priced_bonds.to_excel('priced_bonds.xlsx')